In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
#超参数
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epoch = 2
learing_rate = 0.01

In [3]:
#MNIST 数据集
train_dataset = dsets.MNIST(root='../data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='../data/', train=True, transform=transforms.ToTensor(), download=False)

In [8]:
#加载数据集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
#构建RNN模型
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
rnn = RNN(input_size, hidden_size, num_layers, num_classes)
rnn.cuda()

RNN (
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear (128 -> 10)
)

In [6]:
#定义损失函数和优化函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learing_rate)

In [7]:
#训练网络
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size)).cuda()
        labels = Variable(labels).cuda()
        
        #前向+后向+优化
        optimizer.zero_grad()
        outputs = rnn(images)
        
        #调用损失函数，计算输出值与真实值之间的差
        loss = criterion(outputs, labels)
        
        #后向传播
        loss.backward()
        
        #优化开始
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch+1, num_epoch, i+1, len(train_dataset)//batch_size, loss.data[0]))
        

Epoch [1/2], Step [100/600], Loss: 0.6968
Epoch [1/2], Step [200/600], Loss: 0.2774
Epoch [1/2], Step [300/600], Loss: 0.2221
Epoch [1/2], Step [400/600], Loss: 0.1400
Epoch [1/2], Step [500/600], Loss: 0.3704
Epoch [1/2], Step [600/600], Loss: 0.2101
Epoch [2/2], Step [100/600], Loss: 0.0705
Epoch [2/2], Step [200/600], Loss: 0.0517
Epoch [2/2], Step [300/600], Loss: 0.0291
Epoch [2/2], Step [400/600], Loss: 0.1814
Epoch [2/2], Step [500/600], Loss: 0.2026
Epoch [2/2], Step [600/600], Loss: 0.0936


In [14]:
#测试网络
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size)).cuda()
    
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    
print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 97 %


In [15]:
#保存网络模型
torch.save(rnn.state_dict(), 'rnn.pkl')